In [1]:
import os
with open('../Secrets.env') as f:
    for l in f.readlines():
        if l.strip():
            var, val = l.strip().split("=", 1)
            os.environ[var] = val.replace('"', "")

In [20]:
from requests import Session
from pathlib import Path
import json
import sys
import os

THIS_DIR = Path.cwd()
CORE_DIR = THIS_DIR / "hydrovis" / "Core"
SERVICES_DIR = CORE_DIR / "LAMBDA" / "viz_functions" / "viz_publish_service" / "services"

ESRI_TYPE_TO_DB_TYPES = {
    "esriFieldTypeInteger": ["integer", "bigint"],
    "esriFieldTypeString": ["character varying", "text"],
    "esriFieldTypeDouble": ["double precision", "numeric"],
    "esriFieldTypeGeometry": ["geometry", "USER-DEFINED"],
}
DB_TYPE_TO_ESRI_TYPE = {}
for et, dts in ESRI_TYPE_TO_DB_TYPES.items():
    for dt in dts:
        DB_TYPE_TO_ESRI_TYPE[dt] = et

REFERENCE_DB = "egis"

sys.path.append(str(CORE_DIR / "LAMBDA" / "layers" / "viz_lambda_shared_funcs" / "python"))
from viz_classes import database

sys.path.append(str(CORE_DIR / "Manual_Workflows" / "helper_functions"))
from shared_functions import get_columns, get_schemas

server_services = []
for root, dirs, files in os.walk(SERVICES_DIR):
    for fname in files:
        if fname.endswith('.mapx'):
            fpath = Path(root) / fname
            mapx = json.loads(fpath.read_text())
            changes_made = False
            for layer_def in mapx["layerDefinitions"]:
                layer_name = layer_def["name"]
                if "featureTable" in layer_def and "dataConnection" in layer_def["featureTable"]:
                    data_conn = layer_def["featureTable"]["dataConnection"]
                    if "sqlQuery" in data_conn:
                        sql_query = data_conn["sqlQuery"].lower()
                        full_table_path = sql_query.split(" from ")[1].split(' ')[0].strip()
                        table = full_table_path.split('.')[-1]
                        if '\n' in sql_query:
                            data_conn["sqlQuery"] = sql_query.replace('\n', '')
                            changes_made = True
                            # print(f"{fname} :: \"{layer_name}\" :: sqlQuery string contains newline characters")
                        table = table.strip()
                        if REFERENCE_DB == "egis":
                            if "hydrovis" in full_table_path:
                                schema_table = '.'.join(full_table_path.split('.')[-2:])
                            else:
                                schema_table = f"services.{table}"
                        else:
                            schema_table = f"publish.{table}"
                            
                        # check_schema = full_table_path if REFERENCE_DB == "egis" and "hydrovis" in full_table_path else \
                        # "aep_fim" if "aep_fim" in full_table_path else \
                        #     "reference" if any(x in full_table_path for x in ["reference", "static"]) else \
                        #     "services" if REFERENCE_DB == "egis" else \
                        #     "publish"
                        mapx_qfield_alias_to_db_colname = {}
                        casts = {}
                        columns = [x.strip() for x in sql_query.split("select ")[1].split(" from ")[0].split(',')]
                        for col in columns:
                            col_and_alias = col.lower().split(' as ')
                            orig_col = col_and_alias[0]
                            orig_col_and_cast_type = orig_col.split('::')
                            orig_col = orig_col_and_cast_type[0]
                            qfield_alias = col_and_alias[1] if len(col_and_alias) == 2 else orig_col
                            if len(orig_col_and_cast_type) == 2:
                                casts[qfield_alias] = orig_col_and_cast_type[1]
                            mapx_qfield_alias_to_db_colname[qfield_alias] = orig_col

                        if "queryFields" in data_conn:
                            raw_query_fields = data_conn["queryFields"]
                            qfield_to_raw_index = {}
                            qfield_to_esri_type = {}
                            db_schema_df = get_columns(f'{schema_table}', db_type=REFERENCE_DB)
                            if db_schema_df.empty:
                                print(f"{fname} :: \"{layer_name}\" :: {schema_table} does not exist.")
                                print(full_table_path)
                                continue
                            db_cols_to_dtype = {col: dtype for col, dtype in db_schema_df.values}
                            
                            for i, f in enumerate(raw_query_fields):
                                qfield_to_raw_index[f["name"]] = i
                                qfield_to_esri_type[f["name"]] = f["type"]

                            for qfield, esri_type in qfield_to_esri_type.items():
                                if qfield not in mapx_qfield_alias_to_db_colname:
                                    # print(f"{fname} :: \"{layer_name}\" :: {qfield} is configured in the queryFields section, but is not in the sqlQuery string.")
                                    continue
                                db_col = mapx_qfield_alias_to_db_colname[qfield]
                                if db_col not in db_cols_to_dtype:
                                    # print(f"{fname} :: \"{layer_name}\" :: {check_schema}.{table}.{db_col} is expected in the sqlQuery string, but does not exist in the database.")
                                    continue
                                db_dtype = casts[qfield] if qfield in casts else db_cols_to_dtype[db_col]
                                expected_db_dtypes = ESRI_TYPE_TO_DB_TYPES[esri_type]
                                if db_dtype not in expected_db_dtypes:
                                    # print(f"{fname} :: \"{layer_name}\" :: {check_schema}.{table}.{db_col} ({db_dtype}) is wrongly configured in the queryFields section as {qfield} ({esri_type}).")
                                    new_esri_type = DB_TYPE_TO_ESRI_TYPE[db_dtype]
                                    raw_query_fields[qfield_to_raw_index[qfield]]["type"] = new_esri_type
                                    if new_esri_type == "esriFieldTypeString":
                                        raw_query_fields[qfield_to_raw_index[qfield]]["length"] = 100
                                    if esri_type == "esriFieldTypeString":
                                        del raw_query_fields[qfield_to_raw_index[qfield]]["length"]
                                    changes_made = True

            if changes_made:
                print(f"Writing changes back to mapx: {fname}")
                fpath.write_text(json.dumps(mapx, separators=(',', ' : '), indent=2))



static_stage_based_catfim.mapx :: "Stage-Based CatFIM Sites" :: reference.stage_based_catfim_sites_public does not exist.
hydrovis.reference.stage_based_catfim_sites_public
static_stage_based_catfim.mapx :: "1 ft Invervals Above Action" :: reference.stage_based_catfim_public does not exist.
hydrovis.reference.stage_based_catfim_public
static_stage_based_catfim.mapx :: "Stage-Based CatFIM: Action Threshold" :: reference.stage_based_catfim_public does not exist.
hydrovis.reference.stage_based_catfim_public
static_stage_based_catfim.mapx :: "1 ft Invervals Above Minor" :: reference.stage_based_catfim_public does not exist.
hydrovis.reference.stage_based_catfim_public
static_stage_based_catfim.mapx :: "Stage-Based CatFIM: Minor Threshold" :: reference.stage_based_catfim_public does not exist.
hydrovis.reference.stage_based_catfim_public
static_stage_based_catfim.mapx :: "1 ft Invervals Above Moderate" :: reference.stage_based_catfim_public does not exist.
hydrovis.reference.stage_based_cat

In [ ]:
from shared_functions import sql_to_dataframe

sql_to_dataframe("SELECT DISTINCT nwm_features_flooded_percent FROM services.mrf_gfs_10day_rapid_onset_flooding_hucs_alaska", db_type="egis")